# Hypothesis Testing

## Import Libraries

In [1]:
!pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 5.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import pingouin as pg

## Import Datasets

In [3]:
trx_amount_df = pd.read_csv('/content/drive/MyDrive/Dibimbing/Unified Materials/Hypothesis Testing/trx_amount_df.csv')
payment_duration_df = pd.read_csv('/content/drive/MyDrive/Dibimbing/Unified Materials/Hypothesis Testing/payment_duration_df.csv')
df_trx_abc = pd.read_csv('/content/drive/MyDrive/Dibimbing/Unified Materials/Hypothesis Testing/df_trx_abc.csv')

## Playing around with normal distribution

In [4]:
# IQ follow Normal(100, 15)
# Probability IQ between 100 and 115?
dist = stats.norm(loc=100, scale=15)
prob = dist.cdf(115) - dist.cdf(100)
print(f"Probability IQ between 100 and 115 is {prob:.4f}")

Probability IQ between 100 and 115 is 0.3413


In [6]:
# get probability from Z score
z_score = (115 - 100) / 15
prob = stats.norm.cdf(z_score) - stats.norm.cdf(0)
print(f"Probability IQ between 100 and 115 is {prob:.4f}")

Probability IQ between 100 and 115 is 0.3413


### Exercise

In [7]:
# pr(100_000 < x) from Normal(120_000, 50_000)

In [9]:
# standard normal distribution
# Compute probability of X greater than Z = 1.96


## T-test

### Transaction Amount Data

In [10]:
# snippet of data
trx_amount_df.head()

,control,variant
0,149900,122899
1,124500,233400
2,155900,149400
3,190900,102400
4,120600,187000


In [11]:
# general info
trx_amount_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   control  30 non-null     int64
 1   variant  30 non-null     int64
dtypes: int64(2)
memory usage: 608.0 bytes


In [12]:
# summary statistics
trx_amount_df.describe()

,control,variant
count,30.000000,30.000000
mean,122476.600000,144553.266667
std,35996.270772,41901.599542
min,53500.000000,61800.000000
25%,106375.000000,118075.000000
50%,120600.000000,147100.000000
75%,144400.000000,174500.000000
max,193200.000000,233400.000000


From the mean values: variant > control. Is this difference significant?

In [13]:
# t-test
from statsmodels.stats.weightstats import ttest_ind

df = trx_amount_df

t_stat, p_val, _ = ttest_ind(df['variant'],
                              df['control'])
print(f"t_stat is {t_stat:.4f}")
print(f"p_val is {p_val:.4f}")

t_stat is 2.1890
p_val is 0.0326


p-value < 0.05 -> reject H0: result is significant

### Payment Duration Data

In [14]:
# data snippet
payment_duration_df.head()

,control,variant
0,3.46,4.38
1,4.10,4.14
2,3.56,3.02
3,2.97,3.23
4,3.27,3.37


In [15]:
# general info
payment_duration_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   control  1000 non-null   float64
 1   variant  1000 non-null   float64
dtypes: float64(2)
memory usage: 15.8 KB


In [16]:
# summary statistics
payment_duration_df.describe()

,control,variant
count,1000.00000,1000.000000
mean,3.39082,3.499010
std,0.51680,0.625253
min,1.65000,1.590000
25%,3.05000,3.110000
50%,3.40000,3.470000
75%,3.73250,3.910000
max,4.96000,5.440000


From the mean values: variant > control. Is this difference significant?

In [17]:
# t-test
from statsmodels.stats.weightstats import ttest_ind

df = payment_duration_df

t_stat, p_val, _ = ttest_ind(df['variant'],
                              df['control'])
print(f"t_stat is {t_stat:.4f}")
print(f"p_val is {p_val:.4f}")

t_stat is 4.2176
p_val is 0.0000


p-value < 0.05 -> reject H0: result is significant

## ABC Test with Mean metrics

In [18]:
# data snippet
df_trx_abc.head()

,control,variant1,variant2
0,102400,184899,148000
1,76600,213500,119600
2,152200,114300,279500
3,99000,189700,105800
4,132600,164800,133300


In [19]:
# general info
df_trx_abc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   control   200 non-null    int64
 1   variant1  200 non-null    int64
 2   variant2  200 non-null    int64
dtypes: int64(3)
memory usage: 4.8 KB


In [20]:
# summary statistics
df_trx_abc.describe()

,control,variant1,variant2
count,200.000000,200.000000,200.000000
mean,125963.445000,138620.420000,144345.480000
std,41161.616418,49525.589042,49648.370821
min,13400.000000,-18000.000000,400.000000
25%,99225.000000,110274.250000,110525.000000
50%,130700.000000,137300.000000,142700.000000
75%,156250.000000,173200.000000,175175.000000
max,244800.000000,275300.000000,317800.000000


Turns out there are anomalies in minimum values in variant1, we need to drop it

In [21]:
# how many negative values that need to be dropeed?
df_trx_abc.loc[df_trx_abc.variant1<0].shape[0]

2

In [22]:
# drop them
df_trx_abc = df_trx_abc.loc[df_trx_abc.variant1 >= 0]

In [23]:
# preprocess data
df_trx_abc_melt = df_trx_abc.melt(
    id_vars=None,
    var_name = 'experiment_group',
    value_name='trx_amount'
)
df_trx_abc_melt.head()

,experiment_group,trx_amount
0,control,102400
1,control,76600
2,control,152200
3,control,99000
4,control,132600


In [24]:
df_trx_abc_melt

,experiment_group,trx_amount
0,control,102400
1,control,76600
2,control,152200
3,control,99000
4,control,132600
...,...,...
589,variant2,191900
590,variant2,108100
591,variant2,72900
592,variant2,161000


In [25]:
# one-way ANOVA
anova_res = df_trx_abc_melt.anova(dv='trx_amount', between='experiment_group', detailed=True)
anova_res.loc[:,['Source','SS','DF','MS','F','p-unc']]

,Source,SS,DF,MS,F,p-unc
0,experiment_group,3.748532e+10,2,1.874266e+10,8.804103,0.000171
1,Within,1.258153e+12,591,2.128855e+09,NaN,NaN


Conclusion: Reject H0! There is at least one group that has different transaction amount average!